In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas
import inspect
from datetime import datetime
import seaborn as sns
import glob
from os.path import basename
sns.set_context('poster')
import re
import csv

## Load files

In [3]:
df = pandas.read_excel('../data/globalterrorismdb_0616dist.xlsx')

In [4]:
df_15_16 = pandas.read_excel('../data/Global Terrorism Database 2015-2016 Preliminary Data EMBARGOED.xlsx') 

In [5]:
# is the newest 2015 data same as last?
print df[df.iyear==2015].eventid.count()
print df_15_16[df_15_16.iyear==2015].eventid.count()

14806
14843


In [6]:
#let's use the latest 2015
df = pandas.concat([df[df.iyear < 2015], df_15_16])

Check basic stats

In [ ]:
print df.eventid.count()
print df.iyear.unique()

In [ ]:
df.groupby('iyear').nkill.sum()[36:].sort_values(ascending=False)

## Export cols for 2016

In [ ]:
df[df.iyear==2016].country_txt.value_counts()
print len(set(df[df.iyear==2016].country_txt))

In [ ]:
df[df.iyear==2016][['eventid','iyear','imonth','iday','city','country_txt','nkill','nwound','gname', 'targtype1_txt','weaptype1_txt','summary']].to_csv('../data/2016-for-interactive.csv', encoding='utf8', index=False)

In [ ]:
df.weaptype1_txt

In [ ]:
names = list(set(df.country_txt))

In [ ]:
names.sort(key = len)

## Convert country code

In [ ]:
codes = pandas.read_csv('../data/country_code_cleaned.csv')

In [ ]:
list(codes.name)

In [ ]:
for c in set(df[df.iyear==2016].country_txt):
    if (c in codes.name.values) == False:
        print c, c in codes.name

In [7]:
last_year = df[df.iyear==2016]

In [ ]:
d = dict(zip(list(codes.name), list(codes['alpha-2'])))

In [ ]:
last_year['ISO'] = last_year.country_txt.apply(lambda x: d[x])

In [ ]:
grouped =  last_year.groupby(['imonth','iday','ISO']).size().reset_index(name='count')
grouped.head()

In [9]:
#grouped.apply(lambda x: str(x['imonth']) + '/' + str(x['iday']) + '/2016',axis=1)
grouped['date'] = pandas.to_datetime(2016*10000 + grouped.imonth*100 + grouped.iday, format='%Y%m%d')

NameError: name 'grouped' is not defined

In [ ]:
grouped.pivot_table(index='date', columns='ISO', values='count',  fill_value=0).to_csv("../data/incidents-by-date.csv")

## count by deaths

In [ ]:
grouped =  last_year.groupby(['imonth','iday','ISO']).nkill.sum().reset_index(name='deaths')
grouped['date'] = pandas.to_datetime(2016*10000 + grouped.imonth*100 + grouped.iday, format='%Y%m%d')
grouped.pivot_table(index='date', columns='ISO', values='deaths',  fill_value=0).to_csv("../data/fatalities-by-date.csv")

In [13]:
last_year['date'] = pandas.to_datetime(2016*10000 + last_year.imonth*100 + last_year.iday, format='%Y%m%d')

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [29]:
incidents = last_year.groupby('date').eventid.count().to_frame()

In [30]:
fatalities = last_year.groupby('date').nkill.sum().to_frame()

In [34]:
incidents_keng = pandas.read_csv('../data/keng-incidents-total.csv', index_col=False)
incidents_keng.columns

Index([u'date', u'tt'], dtype='object')

In [26]:
fatalities_keng = pandas.read_csv('../data/keng-fatalities-total.csv', index_col=False)

In [39]:
print list(incidents.eventid) == list(incidents_keng.tt)
print list(fatalities.nkill) == list(fatalities_keng.tt)

True
True
